# Model Server

In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pickle
import mlrun
import os
import requests
import json

In [2]:
def gen_model():
    # Getting the data
    X,y = load_iris(return_X_y=True)
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=123)
    # transforming the data
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    # Getting the model and training it
    classifier = LogisticRegression(random_state = 0, solver='lbfgs', multi_class='auto')
    classifier.fit(X_train, y_train)
    # saving the model
    filename = os.getcwd()+'/model.pkl'
    pickle.dump(classifier, open(filename, 'wb'))
    return X_test,y_test

In [7]:
def test_remote_model_server():
    x,y = gen_model()
    fn = mlrun.import_function('hub://model_server:development').apply(mlrun.auto_mount())
    fn.add_model('iris',model_path = os.getcwd()+'/model.pkl')
    fn.set_envs({'MODEL_CLASS': "ClassifierModel"})
    address = fn.deploy()
    my_dict = {'instances':x.tolist()}
    predict_url = address+"/iris/predict"
    ans = requests.post(url = predict_url,json = my_dict).text
    assert(accuracy_score(y,json.loads(ans)) > 0.8)
    if(accuracy_score(y,json.loads(ans)) > 0.8):
        print('yes')
    else:
        print('no')

In [8]:
test_remote_model_server()

> 2021-08-11 11:57:56,345 [info] Starting remote function deploy
2021-08-11 11:57:56  (info) Deploying function
2021-08-11 11:57:56  (info) Building
2021-08-11 11:57:56  (info) Staging files and preparing base images
2021-08-11 11:57:56  (info) Building processor image
2021-08-11 11:57:58  (info) Build complete
2021-08-11 11:58:21  (info) Function deploy complete
> 2021-08-11 11:58:22,878 [info] function deployed, address=default-tenant.app.dev39.lab.iguazeng.com:30024
yes
